<a href="https://colab.research.google.com/github/jxn1130/kim/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset은 아래 다섯개 활용합니다.
#(dataset/data-02/ 폴더 아래에 있는 데이터 활용)
    - member.csv
    - point_his.csv
    - item_his.csv
    - study_his.csv
    - regdate_his.csv

In [1]:
!pip3 install pyspark

In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

#데이터 읽기
member = spark.read.csv("/content/member.csv", header=True)
point = spark.read.csv("/content/point_his.csv", header=True)
item = spark.read.csv("/content/item_his.csv", header=True)
study = spark.read.csv("/content/study_his.csv", header=True)
regdate = spark.read.csv("/content/regdate_his.csv", header=True)

1. member 테이블에서 idx가 8000과 9000 사이인 row를 찾아주세요.(양 끝값 포함) (6)

In [3]:
#8000에서 9000사이 필터
member.filter(F.col('idx').between(8000,9000)).show()

+----+---+--------+-------+
| idx|sex|  status|  grade|
+----+---+--------+-------+
|8647| 남|유료회원|초4학년|
|8698| 남|유료회원|초2학년|
|8222| 남|유료회원|초4학년|
|8178| 남|유료회원|초1학년|
|8678| 남|유료회원|초4학년|
|8285| 여|유료회원|초4학년|
|8354| 남|유료회원|초3학년|
|8530| 여|유료회원|초3학년|
|8709| 남|유료회원|초1학년|
|8507| 남|유료회원|초2학년|
|8573| 남|유료회원|초3학년|
|8772| 남|유료회원|초3학년|
|8559| 남|유료회원|초3학년|
|8870| 여|유료회원|초5학년|
|8919| 여|유료회원|초4학년|
|8927| 여|유료회원|초3학년|
|8904| 여|유료회원|초5학년|
|8945| 여|유료회원|초6학년|
|8381| 여|유료회원|초4학년|
|8084| 여|유료회원|초6학년|
+----+---+--------+-------+
only showing top 20 rows



2. member 테이블에서 회원 상태별 인원수를 내림차순으로 보여주세요. (6)



In [4]:
#status 기준으로 groupby해서 count하고 내림차순 정렬
member.groupby(F.col('status')).count().orderBy(F.col('count').desc()).show()

+--------+-----+
|  status|count|
+--------+-----+
|유료회원|74198|
|학습만료|  279|
|  재구매|  273|
|    신규|  195|
|    이월|   39|
|    복회|   11|
|    취소|    9|
|이관유료|    8|
|    NULL|    1|
+--------+-----+



3. 학습만료 회원들이 공부한 과목 리스트를 TOP3를 뽑아주세요 (8)



In [5]:
from pyspark.sql.functions import count, desc

#study와 member를 idx기준으로 join
member_study = study.join(member, ['idx'], "inner")

#학습만료인 회원만 filter
expiration = member_study.filter(member_study['status'] == '학습만료')

# 'point nm' 별로 groupby해서 카운트 계산
point_count = expiration.groupBy('pointnm').agg(count('*').alias('count'))

#내림차순 정렬해서 상위 3개만 출력
point_count.orderBy(desc("count")).show(3)

+--------------+-----+
|       pointnm|count|
+--------------+-----+
|    중학 1학년|  115|
|  AI서술형평가|   71|
|AI 구구단 게임|   68|
+--------------+-----+
only showing top 3 rows



4. 등록일이 2023.03.15일인 유료회원들이 가장 많이 착용한 아이템(codename) TOP3를 뽑아주세요 (8)



In [6]:
#member와 item을 idx기준으로 join
member_item = item.join(member, ['idx'], "inner")

#regdate를 추가 join
mir = member_item.join(regdate, ['idx'], "inner")

#조건 필터
paid_0315 = mir.filter((mir["regdate"] == '20230315') & (F.col("status") == "유료회원"))

#codename별로 묶어서 count 내림차순하여 상위 3개 출력
paid_0315.groupby('codename').count().orderBy(F.col('count').desc()).show(3)

+--------+-----+
|codename|count|
+--------+-----+
|    헤어|   31|
|    얼굴|   19|
|    신발|   15|
+--------+-----+
only showing top 3 rows



5. 레벨이 151~160 에 있는 유저들 중 딱 한 명씩만 등록한 날짜들을 구해주세요 (10)

In [7]:
mir_151160 = mir.filter(mir["lv"].between(151,160))

#null값 drop
mir_151160 = mir_151160.dropna(how="any", subset=['regdate'])

# regdate를 기준으로 count
reg_count = mir_151160.groupby(F.col('regdate')).count()

#count가 1인 날짜 출력
reg_count.filter(F.col('count')==1).show()

+--------+-----+
| regdate|count|
+--------+-----+
|20230301|    1|
|20230505|    1|
|20230129|    1|
|20230312|    1|
|20230221|    1|
|20230124|    1|
|20230219|    1|
|20230320|    1|
|20230226|    1|
|20230310|    1|
|20230311|    1|
|20230322|    1|
+--------+-----+



6. item_his 테이블에서 레벨이 null인 유저가 가장 많은 날짜를 구해주세요 (8)



In [8]:
#Null값만 필터
lv_null = item.filter(F.col('lv').isNull())

#count해서 내림차순 정렬하고 1개만 출력
lv_null.groupBy(F.col("proc_ymd")).count().orderBy(F.col('count').desc()).show(1)

+--------+-----+
|proc_ymd|count|
+--------+-----+
|20230504|   10|
+--------+-----+
only showing top 1 row



7. study_his 테이블의 pointnm 컬럼에 대해 공백을 모두 없애주세요 (8)



In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

#변환 함수 작성
def no_space(val):
  return val.replace(" ", "")

#UDF에 등록
no_space_udf = F.udf(no_space, StringType())

# pointnm 열에 UDF 적용
study_nospace = study.withColumn("pointnm", no_space_udf(F.col("pointnm")))

# 결과 확인
study_nospace.show()

+-----+-------+--------+----------+
|  idx|proc_ym|proc_ymd|   pointnm|
+-----+-------+--------+----------+
|88311| 202306|20230628|한글스피치|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
|89542| 202305|20230509| 중학3학년|
|23940| 202305|20230516| 중학1학년|
|23940| 202305|20230516| 중학1학년|
|40502| 202304|20230412| 중학1학년|
| 1741| 202304|20230405| 중학1학년|
|50161| 202304|20230428| 중학1학년|
+-----+-------+--------+----------+
only showing top 20 rows



8. member 테이블의 grade 컬럼에서 숫자만 뽑아 grade라는 컬럼을 재구성해주세요. (초3학년 -> 3) (8)
(UDF, slicing 사용 X)



In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

#grade에서 숫자열 출력
member.withColumn("grade", regexp_extract(F.col("grade"), r"\d", 0)).show()

+-----+---+--------+-----+
|  idx|sex|  status|grade|
+-----+---+--------+-----+
| 6884| 여|유료회원|    3|
| 6331| 남|유료회원|    3|
|69294| 남|유료회원|    5|
|31531| 여|유료회원|    1|
|85784| 여|유료회원|    2|
|58058| 여|유료회원|    3|
|  777| 남|유료회원|    3|
| 5482| 여|유료회원|    2|
|63447| 남|유료회원|    2|
|54957| 남|유료회원|    4|
|55340| 남|유료회원|    2|
|72887| 여|유료회원|    3|
|40008| 여|유료회원|    3|
|57551| 남|유료회원|    5|
|58583| 여|유료회원|    3|
|32219| 남|유료회원|    1|
|13136| 여|유료회원|    4|
|22122| 남|유료회원|    6|
|18306| 남|유료회원|    4|
|28057| 남|유료회원|    3|
+-----+---+--------+-----+
only showing top 20 rows



9. study_his 월별로 pointnm 컬럼에 대한 point 발생 수(count)를 오름차순으로 순번 매겨주세요 (8)



In [11]:
#pointnm Null값 제거
notnull_study = study.filter((F.col("pointnm").isNotNull()) & (F.col("proc_ym").isNotNull()))


# 'point nm' 별로 카운트 계산
point_count = notnull_study.groupBy('proc_ym', 'pointnm').agg(count('*').alias('count'))

#window 변수 생성
#월별로 나눠서 count 오름차순 정렬
window_var = W.Window.partitionBy(F.col("proc_ym")).orderBy("count")

#rank 부여
point_count.withColumn("rank", F.rank().over(window_var)).show()

+-------+-----------------+-----+----+
|proc_ym|          pointnm|count|rank|
+-------+-----------------+-----+----+
| 202304|        중학 특강|   37|   1|
| 202304|        학교 체험|   39|   2|
| 202304| 학교 공부 맛보기|   48|   3|
| 202304|         받아쓰기|   56|   4|
| 202304|       중학 3학년|  135|   5|
| 202304|       중학 2학년|  203|   6|
| 202304|          AI 국어|  211|   7|
| 202304|          AI 영어|  278|   8|
| 202304| 밀크T 지오그래픽|  307|   9|
| 202304|      중학 신입생|  449|  10|
| 202304|      한글 스피치|  478|  11|
| 202304|     자기소개하기|  738|  12|
| 202304|       중학 1학년| 1801|  13|
| 202304|        일기 쓰기| 2247|  14|
| 202304|도전! AI 받아쓰기| 2806|  15|
| 202304|   AI 구구단 게임| 6249|  16|
| 202304|     AI서술형평가|17062|  17|
| 202305|        학교 체험|    7|   1|
| 202305|         받아쓰기|   33|   2|
| 202305|        중학 특강|   37|   3|
+-------+-----------------+-----+----+
only showing top 20 rows



10. pointnm 별로 획득한 point의 종류를 언더바(_)로 이어서 보여주세요 (10)



In [12]:
from pyspark.sql.functions import concat, lit

#point와 study를 idx, proc_ymd 기준으로 join
pointstudy = point.join(study, ['idx', 'proc_ymd'])

#pointnm과 point 사이에 _ 추가한 값을 새로운 열로 추가
pointstudy = pointstudy.withColumn('new_point', concat(F.col('pointnm'), lit('_'), F.col('point')))


# 결과를 출력합니다.
pointstudy.show()

+-----+--------+-------+-----+-------+--------------+-------------------+
|  idx|proc_ymd|proc_ym|point|proc_ym|       pointnm|          new_point|
+-----+--------+-------+-----+-------+--------------+-------------------+
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|20230628| 202306| 2500| 202306|AI 구구단 게임|AI 구구단 게임_2500|
|96826|202

11. point_his의 proc_ymd 컬럼의 날짜 표현 형식을 yyyy-mm-dd 형식이 되도록 바꿔주세요. (8)
(UDF, slicing 사용 X)



In [13]:
from pyspark.sql.functions import to_date, date_format

# 'proc_ymd' 컬럼을 DateType으로 변환하고 yyyy-mm-dd 형식으로 변환
point.withColumn('proc_ymd', date_format(to_date('proc_ymd', 'yyyyMMdd'), 'yyyy-MM-dd')).show()

+-----+-------+----------+-----+
|  idx|proc_ym|  proc_ymd|point|
+-----+-------+----------+-----+
|96465| 202306|2023-06-24| 1000|
|96465| 202306|2023-06-24|  500|
|87940| 202304|2023-04-05| 2000|
|87940| 202304|2023-04-05| 3500|
|87940| 202304|2023-04-05| 4000|
|87940| 202304|2023-04-05| 3000|
|87940| 202304|2023-04-05| 2500|
|88058| 202304|2023-04-05| 3000|
|88058| 202304|2023-04-05| 1000|
|88058| 202304|2023-04-05| 1500|
|88058| 202304|2023-04-05| 2500|
|88058| 202304|2023-04-05|  500|
|88058| 202304|2023-04-05| 2000|
|95844| 202306|2023-06-17| 1500|
|95844| 202306|2023-06-17| 1000|
|95050| 202306|2023-06-12| 3500|
|95050| 202306|2023-06-12| 4000|
|95050| 202306|2023-06-12| 4500|
|92560| 202305|2023-05-11| 1000|
|92560| 202305|2023-05-11|  500|
+-----+-------+----------+-----+
only showing top 20 rows



12. member의 status 컬럼에서 '회원'단어가 있으면 띄어쓰기하고 없으면 띄어쓰기 후 회원 단어 추가해주세요 (유료회원 -> 유효 회원  /  신규 -> 신규 회원) (10)

In [14]:
from pyspark.sql.functions import regexp_replace, concat, lit

# status에서 '회원'을 삭제
member = member.withColumn('status', regexp_replace(member['status'], '회원', ''))

#status 모든 행에 ' 회원'을 추가
member = member.withColumn('status', concat(member['status'], lit(' 회원')))

# 결과를 출력합니다.
member.show()


+-----+---+---------+-------+
|  idx|sex|   status|  grade|
+-----+---+---------+-------+
| 6884| 여|유료 회원|초3학년|
| 6331| 남|유료 회원|초3학년|
|69294| 남|유료 회원|초5학년|
|31531| 여|유료 회원|초1학년|
|85784| 여|유료 회원|초2학년|
|58058| 여|유료 회원|초3학년|
|  777| 남|유료 회원|초3학년|
| 5482| 여|유료 회원|초2학년|
|63447| 남|유료 회원|초2학년|
|54957| 남|유료 회원|초4학년|
|55340| 남|유료 회원|초2학년|
|72887| 여|유료 회원|초3학년|
|40008| 여|유료 회원|초3학년|
|57551| 남|유료 회원|초5학년|
|58583| 여|유료 회원|초3학년|
|32219| 남|유료 회원|초1학년|
|13136| 여|유료 회원|초4학년|
|22122| 남|유료 회원|초6학년|
|18306| 남|유료 회원|초4학년|
|28057| 남|유료 회원|초3학년|
+-----+---+---------+-------+
only showing top 20 rows

